# explore.ipynb

### When this section of the pipeline is done being coded, I will transfer it to ```explore.py```.

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# import necessary packages/modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from math import sqrt

from wrangle import get_zillow_data, prepare_zillow
from preprocessing import zillow_split

# default viz size settings
plt.rc('figure', figsize=(10, 8))
plt.rc('font', size=14)

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
df = get_zillow_data()
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,nan,nan,261.00,nan,nan,nan,nan,14297519,1727539,nan,...,0,0.03,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,nan,nan,261.00,nan,nan,nan,nan,17052889,1387261,nan,...,1,0.06,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,nan,nan,261.00,nan,nan,nan,nan,14186244,11677,nan,...,2,0.01,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,nan,nan,261.00,2.00,nan,nan,nan,12177905,2288172,nan,...,3,-0.10,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,nan,nan,266.00,2.00,nan,nan,1.00,10887214,1970746,nan,...,4,0.01,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [3]:
df.shape

(77580, 69)

In [4]:
df = prepare_zillow(df)
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,regionidcounty,unitcnt,yearbuilt,taxvaluedollarcnt,logerror,propertylandusedesc
parcelid,,,,,,,,,,,,
14297519,3.50,4.00,"3,100.00","6,059.00","33,634,931.00","-117,869,207.00","1,286.00",1.00,"1,998.00","1,023,282.00",0.03,Single Family Residential
17052889,1.00,2.00,"1,465.00","6,111.00","34,449,266.00","-119,281,531.00","2,061.00",1.00,"1,967.00","464,000.00",0.06,Single Family Residential
14186244,2.00,3.00,"1,243.00","6,059.00","33,886,168.00","-117,823,170.00","1,286.00",1.00,"1,962.00","564,778.00",0.01,Single Family Residential
12177905,3.00,4.00,"2,376.00","6,037.00","34,245,180.00","-118,240,722.00","3,101.00",1.00,"1,970.00","145,143.00",-0.10,Single Family Residential
12095076,3.00,4.00,"2,962.00","6,037.00","34,145,202.00","-118,179,824.00","3,101.00",1.00,"1,950.00","773,303.00",-0.00,Single Family Residential


In [5]:
df.columns

Index(['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'fips',
       'latitude', 'longitude', 'regionidcounty', 'unitcnt', 'yearbuilt',
       'taxvaluedollarcnt', 'logerror', 'propertylandusedesc'],
      dtype='object')

In [6]:
df.shape

(52412, 12)

In [7]:
df.isnull().sum()

bathroomcnt                       0
bedroomcnt                        0
calculatedfinishedsquarefeet     82
fips                              0
latitude                          0
longitude                         0
regionidcounty                    0
unitcnt                           0
yearbuilt                       116
taxvaluedollarcnt                 1
logerror                          0
propertylandusedesc               0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52412 entries, 14297519 to 13083743
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   52412 non-null  float64
 1   bedroomcnt                    52412 non-null  float64
 2   calculatedfinishedsquarefeet  52330 non-null  float64
 3   fips                          52412 non-null  float64
 4   latitude                      52412 non-null  float64
 5   longitude                     52412 non-null  float64
 6   regionidcounty                52412 non-null  float64
 7   unitcnt                       52412 non-null  float64
 8   yearbuilt                     52296 non-null  float64
 9   taxvaluedollarcnt             52411 non-null  float64
 10  logerror                      52412 non-null  float64
 11  propertylandusedesc           52412 non-null  object 
dtypes: float64(11), object(1)
memory usage: 5.2+ MB


### Impute the remaining nulls.

### Split the data.